In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime as dt
import seaborn as sns
from scipy import stats
import math

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# 데이터 생성

In [4]:
train = pd.read_excel('/content/drive/My Drive/빅콘폴더/train.xlsx')
train = train[train['판매단가'] < train['취급액']]

In [5]:
train['노출(분)'] = train['노출(분)'].replace(0, math.nan)
train['노출(분)'] = train['노출(분)'].fillna(method='ffill')

In [6]:
test = pd.read_excel('/content/drive/My Drive/빅콘폴더/test.xlsx', header=1)

In [7]:
data = pd.read_excel('/content/drive/My Drive/빅콘폴더/2020_bigcontest_champion_performance.xlsx', header=1)
data = data[data['상품군']!='무형'] 
data = data[data['판매단가'] < data['취급액']]
data['날짜'] = pd.to_datetime(data['방송일시']).dt.date
data['날짜'] = pd.to_datetime(data['날짜'])
data['주문량'] = data['취급액']/data['판매단가']
data["요일"]=data["방송일시"].dt.dayofweek # 요일
data["시"]=data["방송일시"].dt.hour # 시
data['월'] = data['방송일시'].dt.month # 월
weekday = {"월":0,"화":1,"수":2,"목":3,"금":4,"토":5,"일":6}

In [8]:
data = data.fillna(method='ffill')

In [9]:
data = data.iloc[17604:25792]

# 날씨 처리

In [10]:
# 대표 지역들
places = ["강릉","광주","부산","수원","울산","대구","대전","서울","파주"] 
weather = pd.read_excel("/content/drive/My Drive/빅콘폴더/날씨_실제/강릉날씨.xlsx")
row_num = weather.shape[0]
weather["지역"]=["강릉"]*row_num

In [11]:
# 모든 지역 날씨 데이터 세로로 합치기
for p in places[1:]:
    a=pd.read_excel(f"/content/drive/My Drive/빅콘폴더/날씨_실제/{p}날씨.xlsx")
    a["지역"]=[p]*row_num
    weather = pd.concat([weather, a], axis=0)

In [12]:
# "-" -> 0값으로 변경
weather["강수량(mm)"]=weather["강수량(mm)"].replace("-",0)
weather["신적설(cm)"]=weather["신적설(cm)"].replace("-",0)
weather["운량(1/10)"]=weather["운량(1/10)"].replace("-",0)

# object 형 float 형으로 변경
weather["강수량(mm)"]=weather["강수량(mm)"].astype(float)
weather["신적설(cm)"]=weather["신적설(cm)"].astype(float)
weather["운량(1/10)"]=weather["운량(1/10)"].astype(float)

In [13]:
weather = weather.reset_index()

In [14]:
# 날짜 set 맞추기
new_weather = pd.DataFrame()
new_weather = weather.iloc[165:243]
new_weather = pd.concat([new_weather, weather.iloc[753:831]])
new_weather = pd.concat([new_weather, weather.iloc[1341:1419]])
new_weather = pd.concat([new_weather, weather.iloc[1929:2007]])
new_weather = pd.concat([new_weather, weather.iloc[2517:2595]])
new_weather = pd.concat([new_weather, weather.iloc[3105:3183]])
new_weather = pd.concat([new_weather, weather.iloc[3693:3771]])
new_weather = pd.concat([new_weather, weather.iloc[4281:4359]])
new_weather = pd.concat([new_weather, weather.iloc[4869:4947]])

In [15]:
# 서울 날씨와 강릉, 부산 날씨로 칼럼별 주문량 상관성 확인 및 지역별 날씨의 영향도 확인
gang_weather = new_weather.loc[new_weather["지역"]=="강릉",:]
seoul_weather = new_weather.loc[new_weather["지역"]=="서울",:]
busan_weather = new_weather.loc[new_weather["지역"]=="부산",:]
suwon_weather = new_weather.loc[new_weather["지역"]=="수원",:]
sudogwon_weather = new_weather.loc[(new_weather["지역"]=="서울")|(weather["지역"]=="수원"),:]

In [16]:
day_data = data.copy()

In [17]:
day_data['날짜'] = pd.to_datetime(day_data['날짜'])
new_weather['날짜'] = pd.to_datetime(new_weather['날짜'])

In [18]:
all_data = day_data.merge(new_weather, on='날짜')
all_data = all_data.groupby(['날짜']).mean()

In [19]:
sudogwon_weather['날짜'] = pd.to_datetime(sudogwon_weather['날짜'] )
seoul_weather['날짜'] = pd.to_datetime(seoul_weather['날짜'] )
busan_weather['날짜'] =pd.to_datetime(busan_weather['날짜'] )
suwon_weather['날짜'] = pd.to_datetime(suwon_weather['날짜'] )
gang_weather['날짜'] = pd.to_datetime(gang_weather['날짜'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [20]:
sudogwon_data = day_data.merge(sudogwon_weather, on="날짜")
suwon_data = day_data.merge(suwon_weather, on="날짜")
seoul_data = day_data.merge(seoul_weather, on="날짜")
busan_data = day_data.merge(busan_weather, on="날짜")
gang_data = day_data.merge(gang_weather, on="날짜")

In [21]:
fc_prop = pd.read_csv("/content/drive/My Drive/빅콘폴더/날씨예보/서울날씨/강수확률.csv")
fc_amount = pd.read_csv("/content/drive/My Drive/빅콘폴더/날씨예보/서울날씨/6시간 강수량.csv") 
fc_sky = pd.read_csv("/content/drive/My Drive/빅콘폴더/날씨예보/서울날씨/하늘상태.csv")
fc_top = pd.read_csv("/content/drive/My Drive/빅콘폴더/날씨예보/서울날씨/일최고기온.csv")
fc_bottom = pd.read_csv("/content/drive/My Drive/빅콘폴더/날씨예보/서울날씨/일최저기온.csv")

In [22]:
forecast_data = {"강수확률":fc_prop, "강수량":fc_amount,"하늘상태":fc_sky, "일최고기온":fc_top, "일최저기온":fc_bottom}

In [23]:
forecast = pd.DataFrame()

for name, fcdata in forecast_data.items():
    amount_list=[]
    cycle = (fcdata.shape[0])//365 # 하루 관측량
    for i in range(365):
        day_weather = fcdata.iloc[i*cycle:(i+1)*cycle,-1].mean()
        amount_list.append(day_weather)
    forecast[name]=amount_list

In [24]:
forecast = forecast.iloc[165:243]

In [25]:
# 날짜 칼럼 추가
forecast["날짜"]=train["날짜"].unique()

# 마더코드, 일/월별 처리

In [37]:
MomCodeGroup2 = train.groupby(['마더코드']).sum()

In [38]:
MomCodeGroup2['주문량/노출'] = MomCodeGroup2['주문량']/MomCodeGroup2['노출(분)']
MomCodeGroup2.sort_values('주문량/노출', ascending=False).head()

,Unnamed: 0,노출(분),상품코드,판매단가,취급액,요일,시,주문량,holiday,월,holiday(includeSS),실제_서울_최고기온,실제_서울_최저기온,실제_서울_강수량(mm),실제_서울_평균풍속(m/s),실제_수원_최고기온,실제_수원_최저기온,실제_수원_강수량(mm),실제_수원_평균풍속(m/s),실제_파주_최고기온,실제_파주_최저기온,실제_파주_강수량(mm),실제_파주_평균풍속(m/s),예보_서울_강수확률,예보_서울_강수량,예보_서울_하늘상태,예보_서울_일최고기온,예보_서울_일최저기온,예보_일산_강수확률,예보_일산_강수량,예보_일산_일최고기온,예보_일산_일최저기온,예보_수원_강수확률,예보_수원_강수량,예보_수원_일최고기온,예보_수원_일최저기온,계절,DATE,미세먼지(㎍/㎥),오존(ppm),이산화질소농도(ppm),일산화탄소농도(ppm),아황산가스농도(ppm),초미세먼지(㎍/㎥),MONTH,HOUR,주문량/노출
마더코드,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
100492,75615,60.0,604461,149700,286104000,9,48,5733.547094,0,24,0,79.2,60.3,110.7,6.3,81.3,60.0,45.9,6.6,78.9,53.4,46.2,5.7,44.797297,2.763158,7.033784,82.578947,59.2500,28.175676,0.394737,80.526316,54.5625,40.135135,1.578947,82.578947,58.6875,6,60572487,83.88,60.24,0.10620,0.06084,1.500,0.01080,24,48,95.559118
100499,145080,120.0,1209006,239400,420256000,15,117,10532.731830,0,48,0,186.9,138.6,0.3,11.1,184.2,138.0,3.0,12.3,181.2,122.7,5.1,9.6,137.229730,6.907895,18.324324,183.315789,139.8750,120.000000,5.526316,177.000000,132.9375,133.986486,6.118421,180.473684,136.8750,12,121144908,136.44,83.28,0.19428,0.10164,2.532,0.02136,48,117,87.772765
100046,151203,60.0,1200756,185400,162001000,18,90,5242.750809,0,48,0,158.4,120.6,221.4,12.6,162.6,120.0,91.8,13.2,157.8,106.8,92.4,11.4,89.594595,5.526316,14.067568,165.157895,118.5000,56.351351,0.789474,161.052632,109.1250,80.270270,3.157895,165.157895,117.3750,12,121144974,167.76,120.48,0.21240,0.12168,3.000,0.02160,48,90,87.379180
100754,76092,60.0,606609,119700,187163000,15,39,4690.802005,0,24,3,86.7,55.5,0.0,5.4,83.1,52.2,0.0,5.1,83.1,46.5,0.0,2.7,66.689189,0.000000,9.668919,87.000000,60.9375,52.702703,0.000000,84.631579,54.1875,72.770270,0.394737,85.736842,59.2500,6,60572493,49.68,28.08,0.09552,0.05016,1.152,0.01020,24,39,78.180033
100323,151497,120.0,1208934,299400,452290000,21,99,9063.927856,0,48,3,177.3,126.3,0.0,11.4,171.3,120.3,0.0,10.2,170.4,107.4,0.0,6.6,142.297297,11.250000,18.729730,170.526316,124.6875,118.378378,12.631579,165.789474,114.9375,145.337838,11.447368,169.263158,122.0625,12,121144977,138.12,85.56,0.21396,0.11016,2.520,0.02136,48,99,75.532732


In [39]:
topcode = list(MomCodeGroup2.sort_values('주문량/노출', ascending=False).index[:50])

# 모델학습 데이터 생성 및 분리

In [28]:
y_train = train['주문량']
x_train = train.drop('주문량',axis=1)

In [30]:
#모든 카테고리에 대해 따로따로 자동 변수생성
category_list = x_train['상품군'].unique().tolist()
for i in category_list:
  globals()['x_train_{}'.format(i)] = x_train.loc[x_train['상품군']==i] #카테고리에 해당하는 주문량제외 train데이터 할당
  globals()['y_train_{}'.format(i)] = train['주문량'].loc[x_train['상품군']==i] #카테고리에 해당하는 train의 주문량을 할당

In [31]:
for i in range(11):
  print('x_train_{}'.format(category_list[i])) #11개 상품군 다 성공적으로 변수 생성됨.

x_train_inner
x_train_cloth
x_train_elec
x_train_living
x_train_food
x_train_health
x_train_bedding
x_train_furniture
x_train_beauty
x_train_etc
x_train_life


In [33]:
#카테고리별로 x,y를 train test 스플릿해줌.
from sklearn.model_selection import train_test_split
for i in category_list:
  x = globals()['x_train_{}'.format(i)]
  y = globals()['y_train_{}'.format(i)]
  globals()['X_train_{}'.format(i)], globals()['X_test_{}'.format(i)], globals()['Y_train_{}'.format(i)], globals()['Y_test_{}'.format(i)] = train_test_split(x, y, test_size=0.33, random_state=0)
# 카테고리별로 X_train_카테고리 , X_test_카테고리, Y_train_카테고리, Y_test_카테고리 생성됐음.

# 인코딩

In [34]:
def add_frequency_encoding(data,column): #데이터프레임,열을 받아서 빈도수인코딩열을 추가해줌
  enc_nom = (data.groupby(column).size())/len(data)
  data['freq_encode_{}'.format(column)] = data[column].apply(lambda x:enc_nom[x])
  print("freq_encode column was added")

In [35]:
# 4.카테고리별 분리해서 빈도수 인코딩
for i in range(11):
  hi_train = pd.DataFrame()
  hi_test = pd.DataFrame()
  hi_train =  globals()['X_train_{}'.format(category_list[i])] #전역변수중 X_train_으로 시작하는 카테고리변수를 찾아서 할당
  hi_test = globals()['X_test_{}'.format(category_list[i])]
  add_frequency_encoding(hi_train,'분류')
  add_frequency_encoding(hi_test,'분류')  # X_train_카테고리 X_test_카테고리 별로 '분류' 빈도인코딩열 생성

freq_encode column was added
freq_encode column was added
freq_encode column was added
freq_encode column was added
freq_encode column was added
freq_encode column was added
freq_encode column was added
freq_encode column was added
freq_encode column was added
freq_encode column was added
freq_encode column was added
freq_encode column was added
freq_encode column was added
freq_encode column was added
freq_encode column was added
freq_encode column was added
freq_encode column was added
freq_encode column was added
freq_encode column was added
freq_encode column was added
freq_encode column was added
freq_encode column was added


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [40]:
# 4번. 탑코드 적용
for i in range(11):
  hi_train = pd.DataFrame()
  hi_test = pd.DataFrame()
  hi_train =  globals()['X_train_{}'.format(category_list[i])] #전역변수중 X_train_으로 시작하는 카테고리변수를 찾아서 할당
  hi_test =  globals()['X_test_{}'.format(category_list[i])]
  globals()['X_train_{}'.format(category_list[i])]['topcode'] = hi_train['마더코드'].isin(topcode)
  globals()['X_test_{}'.format(category_list[i])]['topcode'] = hi_test['마더코드'].isin(topcode)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


# 모델 생성 및 학습

In [41]:
from sklearn.ensemble import RandomForestRegressor
forest = RandomForestRegressor()

In [56]:
feature_selection = X_train_beauty.iloc[:,[2,7,11,13,14,31,32,33,34,36,37,38,40,41,42,54,56,57]].columns.values
feature_selection

array(['노출(분)', '판매단가', '요일', 'holiday', '월', '예보_서울_강수확률', '예보_서울_강수량',
       '예보_서울_하늘상태', '예보_서울_일최고기온', '예보_일산_강수확률', '예보_일산_강수량',
       '예보_일산_일최고기온', '예보_수원_강수확률', '예보_수원_강수량', '예보_수원_일최고기온', 'HOUR',
       'freq_encode_분류', 'topcode'], dtype=object)

In [43]:
for i in category_list:
  globals()['shop_train_x_{}'.format(i)] = globals()['X_train_{}'.format(i)][feature_selection]
  globals()['shop_test_x_{}'.format(i)] = globals()['X_test_{}'.format(i)][feature_selection]

In [58]:
forest.fit(shop_train_x_beauty, Y_train_beauty)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [59]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [60]:
predictions = forest.predict(shop_train_x_beauty)
mape = mean_absolute_percentage_error(Y_train_beauty, predictions)
mape

28.114938388674958

In [48]:
some_data = X_train_beauty.iloc[:8] # 8개의 Data의 주문량 예측해 보겠음.
some_labels = Y_train_beauty.iloc[:8]
some_data = some_data[feature_selection]
print("예측", forest.predict(some_data))
print("레이블:", list(some_labels))

예측 [ 810.17933185  863.88310775  163.38309069 1133.57646449  727.82376147
  277.1864928   899.89756187  792.58945656]
레이블: [828.170426065162, 570.418060200669, 73.859649122807, 1348.49246231155, 587.51256281407, 323.991416309012, 1147.61306532663, 1022.40677966101]


#모든 카테고리 반복문으로 한번에 fit & predict 결과

In [77]:
mape_score = []
for i in category_list:
  num = 0
  category_data = globals()['shop_train_x_{}'.format(i)]
  category_label =  globals()['Y_train_{}'.format(i)]
  forest.fit(category_data,category_label)
  predictions = forest.predict(category_data)
  mape_score.append(mean_absolute_percentage_error(category_label, predictions))

MAPE score of : inner is 40.08985627236814
MAPE score of : cloth is 40.08985627236814
MAPE score of : elec is 40.08985627236814
MAPE score of : living is 40.08985627236814
MAPE score of : food is 40.08985627236814
MAPE score of : health is 40.08985627236814
MAPE score of : bedding is 40.08985627236814
MAPE score of : furniture is 40.08985627236814
MAPE score of : beauty is 40.08985627236814
MAPE score of : etc is 40.08985627236814
MAPE score of : life is 40.08985627236814


In [79]:
for i in range(len(category_list)):
  print("MAPE score of:",category_list[i],":",mape_score[i])

MAPE score of: inner : 40.08985627236814
MAPE score of: cloth : 38.64075528697359
MAPE score of: elec : 45.85126008822423
MAPE score of: living : 55.131065616503825
MAPE score of: food : 30.70625134858927
MAPE score of: health : 31.232663852208674
MAPE score of: bedding : 45.874644143272455
MAPE score of: furniture : 72.36153065894143
MAPE score of: beauty : 28.443297937440242
MAPE score of: etc : 66.86787025251958
MAPE score of: life : 45.576192193013476
